In [1]:
import sys
sys.path.append("../")
import utils
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import PchipInterpolator as pchip
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as rmse, mean_absolute_percentage_error as mape, pairwise_distances, r2_score
from scipy.optimize import curve_fit
import matplotlib.patches as patches

plt.rcParams.update({'font.size': 15})

# TRAINING DATA

**Prognosis dataset** 

Consists of duty cycles with one voltage curve for every 200 cycles (one every 10 cycles for cycles 1-200)


There is a diagnostic dataset containing all possible degradations for the simulated cells. Each duty cycle of the prognostic dataset is constructed from a selection on the evolution of the voltage curves, which are obtained from the diagnostic dataset.


Data structures:
- cyc: array, 35 cycles
	- cycles from 0 to 200 with a step of 10 (21)
	- cycles from 200 to 3000 with a step of 200 (15)
- Q: array, 1001 samples. Capacity curve.
- QL: array, num_prognosis_samples x 35 cycles
- V: array, num_diagnosis_samples x 1001. Voltage curves (from the diagnosis dataset).
- indexes: array, num_prognosis_samples samples x 35 cycles. This array contains for every pair (sample, cycle) the index of the diagnosis dataset where the sample is. If the capacity loss associated with the sample exceeds 80% of the initial capacity, the index is negative (nan).

## Preprocessing

1. Load the data

In [2]:
chemistry = 'LFP' # 'LFP', 'NCA' or 'NMC'

cyc = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 
400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000] # cycles for which there is data
# as the cycles are not equally spaced, we need to select the cycles we want to use
selected_cycles = [0, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000]
# the index of the selected cycles are looked up in the original array
selected_cycles_index = np.array([np.where(np.array(cyc)==el)[0][0] for el in selected_cycles])

# --------------------- Indices to get data from diagnosis dataset --------------------
indices = utils.read_mat('../data/'+chemistry+'/Vi.mat')['Vi'][:,selected_cycles_index] # indices where to look for the voltage curve in the diagnosis dataset
indices = indices - 1 # in matlab, indices start at 1, in python they start at 0
indices = indices.astype(np.int32) # change dtype of indices to int -> now nans are negative numbers
    
print("Total samples: ", indices.shape[0])

# ----------------- Degradation modes and capacity loss---------------
degradations = utils.read_mat('../data/'+chemistry+'/path_prognosis.mat')['path_prognosis'].T[:,selected_cycles_index]
degradation_modes = degradations[:,:,0:3].astype(np.float32)
QL = degradations[:,:,3].astype(np.float32)
QL[QL==0] = 80 # needed in order to check if the QL is increasing
QL[:,0] = 0 # there is not any capacity loss in the first cycle
assert indices.shape[0] == QL.shape[0] == degradation_modes.shape[0]

# --------------- Voltage and Capacity --------------
V = utils.read_mat_hdf5('../data/'+chemistry+'/volt.mat', 'volt')
Q = utils.read_mat("../data/Q.mat")['Qnorm'].flatten()

Total samples:  127662


2. Remove duplicated samples

In [3]:
# some samples are duplicated
_, unique_indices = np.unique(indices, return_index=True, axis=0) # remove duplicates
indices = indices[unique_indices]
QL = QL[unique_indices]
degradation_modes = degradation_modes[unique_indices]
print("Total samples after removing duplicates: ", indices.shape[0])
assert indices.shape[0] == QL.shape[0] == degradation_modes.shape[0]

Total samples after removing duplicates:  66097


3. Remove samples that gain capacity over time

In [4]:
# some samples gain capacity over time, we remove them
valid_indices = utils.get_increasing_samples(QL)
indices = indices[valid_indices]
QL = QL[valid_indices]
degradation_modes = degradation_modes[valid_indices]
print("Total samples after removing samples that gain capacity: ", indices.shape[0])

Total samples after removing samples that gain capacity:  65958


4. Calculate IC curves

In [5]:
IC_curves = utils.retrieve_curves(chemistry, indices, V, Q)

# Calculate the minimum and maximum values
min_val = np.min(IC_curves)
max_val = np.max(IC_curves)

# Normalize using the minimum and maximum values
IC_curves = (IC_curves - min_val) / (max_val - min_val)

assert indices.shape[0] == QL.shape[0] == degradation_modes.shape[0] == IC_curves.shape[0]

5. Label knees

In [6]:
knee_labels = utils.label_knees(QL, degradation_modes)

6. Apply sliding windows and get pairs X, y

In [7]:
window_size_cycle = 800 # number of cycles to use for the windows size
# get position of selected cycles where cycle = window_size_cycle
window_size = selected_cycles.index(window_size_cycle) + 1

x_ICs = []
y_degradation_modes = []
y_QLs = []
y_knees = []
sample_index = []

num_samples, num_cycles, series_len = IC_curves.shape

for sample_num in range(10): # this takes long, just illustrates what happens with the first 10 samples
    for cycle_num in range(num_cycles-window_size+1):
        ICs_cur = IC_curves[sample_num, cycle_num:cycle_num+window_size, :]
        degradation_modes_cur = degradation_modes[sample_num, cycle_num:cycle_num+window_size, :]
        QLs_cur = QL[sample_num, cycle_num:cycle_num+window_size]
        knee_cur = knee_labels[sample_num][cycle_num:cycle_num+window_size]
        # if any of the degradation modes is nan or degradation is 0.8 or there is no IC curve skip the sample
        if np.any(np.isnan(degradation_modes_cur)) or np.any(QLs_cur == 80) or np.any(ICs_cur == 0.0):
            break
        else:
            x_ICs.append(ICs_cur)
            y_degradation_modes.append(degradation_modes_cur)
            y_QLs.append(QLs_cur)
            y_knees.append(knee_cur)
            sample_index.append(sample_num)

x_ICs = np.array(x_ICs)
y_degradation_modes = np.array(y_degradation_modes)
y_QLs = np.array(y_QLs)
y_knees = np.array(y_knees)
sample_index = np.array(sample_index)
assert x_ICs.shape[0] == y_degradation_modes.shape[0] == y_QLs.shape[0] == y_knees.shape[0] == sample_index.shape[0]

7. Remove samples in which the IC curves are the same

In [8]:
_, unique_indices = np.unique(x_ICs, return_index=True, axis=0)
# update the arrays
x_ICs = x_ICs[unique_indices]
y_degradation_modes = y_degradation_modes[unique_indices]
y_QLs = y_QLs[unique_indices]
y_knees= y_knees[unique_indices]
sample_index = sample_index[unique_indices]
print(x_ICs.shape)
print(y_degradation_modes.shape)
print(y_QLs.shape)
print(y_knees.shape)
print(sample_index.shape)

(28, 5, 128)
(28, 5, 3)
(28, 5)
(28, 5)
(28,)
<class 'numpy.ndarray'>


## Load dataset

In [196]:
chemistry = 'LFP' # 'LFP', 'NCA' or 'NMC'
selected_cycles = [0, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000]

with h5py.File('../data/'+chemistry+'/datasets.h5', 'r') as h5f:
    # Load the first dataset
    first_dataset = h5f['dataset_pre']
    IC_curves = first_dataset['IC_curves'][()]
    QLs = first_dataset['QLs'][()]
    degradation_modes = first_dataset['degradation_modes'][()]

    second_dataset = h5f['dataset']
    x_ICs_train = second_dataset['x_ICs_train'][()]
    y_degradation_modes_train = second_dataset['y_degradation_modes_train'][()]
    y_QLs_train = second_dataset['y_QLs_train'][()]
    y_silent_modes_train = second_dataset['y_silent_modes_train'][()].astype(int)
    sample_index_train = second_dataset['sample_index_train'][()]

#del(first_dataset)
#del(second_dataset)

In [10]:
chemistry = 'LFP'
path = '../data/'+chemistry
    
with h5py.File(path+'/datasets.h5', 'r') as h5f:
	first_dataset = h5f['dataset_pre']
	norm_values = first_dataset['norm_values'][()]
	min_val = norm_values[0]
	max_val = norm_values[1]

	dataset = h5f['dataset']
	x_train = dataset['x_ICs_train'][()]
	y_regression_train = dataset['y_degradation_modes_train'][()]
	y_classification_train = dataset['y_silent_modes_train'][()].astype(np.float32)
del(first_dataset)
del(dataset)

print(x_train.shape)
print(y_regression_train.shape)
print(y_classification_train.shape)

(187702, 5, 128)
(187702, 5, 3)
(187702,)
